In [1]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt

import keras, os
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2

Using TensorFlow backend.


In [2]:
train_path = "../dataset/train.csv"
bbox_path = "../dataset/bb_labels.csv"

In [105]:
class bbox_data():
    def __init__(self, batch_size, width, height):
        self.index = 0
        self.batch_size = batch_size
        self.width = width
        self.height = height
        self.read_data(train_path, bbox_path)
        
    def read_data(self, train_path, bbox_path):
        data = np.array(pd.read_csv(train_path))
        bbox = np.array(pd.read_csv(bbox_path))
        self.bbox = bbox
        self.train = data
    def display_bbox(self):
        return self.bbox
    def count_location(self, height, width, x_min, y_min, x_max, y_max):
        target_y_min = (y_min*self.height) / height
        target_x_min = (x_min*self.width) / width
        target_y_max = (y_max*self.height) / height
        target_x_max = (x_max*self.width) / width
        return target_x_min, target_y_min, target_x_max, target_y_max
        
    def next_batch(self):
        image_dir = "../dataset/train/"
        image = []
        label = []
        for i in range(self.batch_size):
            idx = (self.index + i)%(self.bbox.shape[0])
            img = cv2.imread(image_dir+self.bbox[idx][0])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img[0:self.bbox[idx][6], 0:self.bbox[idx][5]]
            img = cv2.resize(img,(self.height, self.width))
            image.append(img)
            
            x_min, y_min, x_max, y_max = self.count_location(self.bbox[idx][6], self.bbox[idx][5], self.bbox[idx][1], self.bbox[idx][2], self.bbox[idx][3], self.bbox[idx][4])
            label.append([x_min, y_min, x_max, y_max])
        self.index += self.batch_size
        return np.array(image), np.array(label)
        
#     def has_next_batch(self):
#         if (self.index + self.batch_size > self.bbox.shape[0]):
#             return False
#         return True

#     def reset_batch(self):
#         self.index = 0
        

In [4]:
def CNN():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=(256, 256, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4))
    model.summary()
    return model

In [5]:
def main():
    data = bbox_data(10, 256, 256)
    model = CNN()
    model.compile(loss='mse', optimizer='adam')
    earlystopping = EarlyStopping(monitor='val_loss', patience=1, mode='min')
    checkpoint = ModelCheckpoint(filepath="model.h5", save_best_only=True, 
                                 save_weights_only=True, monitor='val_loss', mode='min')
    for epoch in range(50):
        for batch_size in range(50):
            x, y = data.next_batch()
            
            model.fit(x, y, batch_size = 10, epochs = 1 , validation_split=0.02) #20
    return model

In [6]:
model = main()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
con

9/9 [==============================] - 0s - loss: 18590.3516 - val_loss: 11004.4688
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 16932.5078 - val_loss: 10469.8057
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 8147.8262 - val_loss: 2801.0112
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 2510.5044 - val_loss: 745.7147
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 14844.3213 - val_loss: 896.6495
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 4044.8784 - val_loss: 3573.0801
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1657.8414 - val_loss: 2041.8794
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 4356.3071

9/9 [==============================] - 0s - loss: 2283.2388 - val_loss: 868.2758
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 2281.3154 - val_loss: 1415.3806
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 2797.9343 - val_loss: 443.9039
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1065.6443 - val_loss: 4733.2446
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1291.8856 - val_loss: 313.9940
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1205.0275 - val_loss: 972.2525
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 2465.4058 - val_loss: 2698.3374
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 779.6907 - val_l

9/9 [==============================] - 0s - loss: 1542.1847 - val_loss: 6712.1138
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1451.4178 - val_loss: 1419.4255
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1353.5277 - val_loss: 1307.6823
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 920.7764 - val_loss: 431.0739
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1041.2883 - val_loss: 2773.3040
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1539.6223 - val_loss: 1093.9795
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1188.1766 - val_loss: 154.8599
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1556.8446 - val

9/9 [==============================] - 0s - loss: 1214.3020 - val_loss: 1402.0685
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1452.3950 - val_loss: 605.1158
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 896.6994 - val_loss: 21.0204
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1517.2206 - val_loss: 992.6349
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1260.0293 - val_loss: 3633.6304
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1737.5972 - val_loss: 686.9605
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1845.3346 - val_loss: 1319.5571
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 1154.0031 - val_lo

...

9/9 [==============================] - 0s - loss: 81.2600 - val_loss: 18.9259
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 79.7459 - val_loss: 30.3073
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 88.1112 - val_loss: 167.7276
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 55.8850 - val_loss: 1640.1952
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 195.8334 - val_loss: 464.7268
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 94.8888 - val_loss: 363.5657
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 58.2204 - val_loss: 113.0297
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 37.5455 - val_loss: 28.1905
Train

9/9 [==============================] - 0s - loss: 327.5365 - val_loss: 96.2327
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 182.7246 - val_loss: 613.3005
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 154.2951 - val_loss: 102.3019
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 186.8982 - val_loss: 134.3067
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 58.8873 - val_loss: 46.2149
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 121.6456 - val_loss: 118.5003
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 80.6919 - val_loss: 179.4390
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 90.7517 - val_loss: 20.2808
Tr

9/9 [==============================] - 0s - loss: 101.3014 - val_loss: 26.8371
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 69.5228 - val_loss: 37.0915
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 95.6240 - val_loss: 67.3074
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 114.3533 - val_loss: 22.6649
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 42.4010 - val_loss: 185.4086
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 92.1262 - val_loss: 17.9587
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 115.3875 - val_loss: 99.3967
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 80.3969 - val_loss: 44.9623
Train on

9/9 [==============================] - 0s - loss: 103.2970 - val_loss: 43.2404
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 98.3483 - val_loss: 24.7535
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 72.5889 - val_loss: 266.5572
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 99.3170 - val_loss: 40.1935
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 91.1367 - val_loss: 39.0104
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 187.7591 - val_loss: 76.4764
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 586.6653 - val_loss: 13.2772
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 107.5047 - val_loss: 65.2188
Train o

9/9 [==============================] - 0s - loss: 86.4193 - val_loss: 32.0767
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 66.9762 - val_loss: 670.0905
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 121.4289 - val_loss: 171.1939
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 598.7643 - val_loss: 39.4762
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 107.6528 - val_loss: 64.2926
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 109.6498 - val_loss: 28.7189
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 123.5128 - val_loss: 29.0183
Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 0s - loss: 88.8249 - val_loss: 12.9291
Train

In [140]:
class test_data():
    def __init__(self, batch_size, height, width):
        self.index = 0
        self.next_round = False
        self.end = False
        self.batch_size = batch_size
        self.all_size = 0
        self.height = height
        self.width = width
        self.read_data()
        
    def read_data(self):
        self.all_img_list = np.array(os.listdir("../dataset/test/"))
        self.train_img_list = np.array(pd.read_csv("../dataset/bb_labels.csv"))[:,0]
    def next_batch(self):
        image_dir = "../dataset/test/"
        image = []
        img_size = []
        file_name = []
        if (self.next_round == True):
            start = self.index
            end = self.all_img_list.shape[0]
            self.end = True
        else:
            start = self.index
            end = self.index+self.batch_size
        for img_name in self.all_img_list[start:end]:
            if (img_name not in self.train_img_list):
                file_name.append(img_name)
                img = cv2.imread(image_dir+img_name)
                height, width, channels = img.shape
                img_size.append([height, width])
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (self.height, self.width))
                image.append(img)
        self.index = self.index + self.batch_size
        return np.array(image), np.array(img_size), np.array(file_name)
        
    def has_next(self):
        if (self.end):
            return False
        if (self.index + self.batch_size >= self.all_img_list.shape[0]):
            self.next_round = True
        return True
            

In [141]:
def count_location(target_point, ori_img_size):
    origin_point = []
    for i in range(target_point.shape[0]):
        x_min = int(ori_img_size[i][1] * target_point[i][0] / 256)
        x_max = int(ori_img_size[i][1] * target_point[i][2] / 256)
        y_min = int(ori_img_size[i][0] * target_point[i][1] / 256)
        y_max = int(ori_img_size[i][0] * target_point[i][3] / 256)
        origin_point.append([x_min, y_min, x_max, y_max])
    return np.array(origin_point)
    

In [142]:
def predict(model):
    all_point = []
    all_name = []
    data = test_data(10, 256, 256)
    count = 0
    while(data.has_next):
        x, img_size, file_name = data.next_batch()
        if (x.shape[0] == 0 ):
            break
        pred = model.predict(x)
        count += x.shape[0]
        ori_point = count_location(pred, img_size)
        all_point.append(ori_point)
        all_name.append(file_name)
    return np.array(all_point), np.array(all_name)
        

In [143]:
all_point, all_name = predict(model)

In [144]:
csv_point = []
csv_name = []
for i in range(all_point.shape[0]):
    for j in range(all_point[i].shape[0]):
        csv_point.append(all_point[i][j])
        csv_name.append(all_name[i][j])
csv_point = np.array(csv_point)
csv_name = np.expand_dims(np.array(csv_name), 1)

In [130]:
# train = bbox_data(10, 256, 256)
# train = train.display_bbox()
# train = train[:,0:5]

In [131]:
# train.shape

(599, 5)

In [145]:
new_train = np.concatenate((csv_name, csv_point),axis=1)

In [146]:
new_train.shape

(15610, 5)

In [134]:
# all_train = np.concatenate((train, new_train), axis=0)

In [135]:
# all_train.shape

(9850, 5)

In [147]:
df = pd.DataFrame(new_train, columns=['image', 'xmin', 'ymin', 'xmax', 'ymax'])

In [148]:
df

,image,xmin,ymin,xmax,ymax
0,35b287bc.jpg,25,144,1062,486
1,ee6fd7fc.jpg,196,216,921,422
2,021f07cc.jpg,30,39,1006,394
3,004cd940.jpg,142,70,882,454
4,3cd6fa25.jpg,14,147,1038,513
5,c6de08c5.jpg,21,116,1023,352
6,9b01c4fe.jpg,24,149,1047,418
7,621f8214.jpg,7,19,1101,680
8,8ebbfbf1.jpg,3,9,686,347
9,72d642bc.jpg,15,-8,1061,411


In [149]:
df.to_csv("bbox_label_test.csv")

In [150]:
model.save("model.h5")